In [63]:
import pandas as pd
import os
from bs4 import BeautifulSoup


In [2]:
promising = pd.read_csv('/Users/caged/Dropbox/RSN Mining/promising indicators.csv')
promising.head()

,orig_id,description,new_id
0,1b,Specifically note that suppliers may continue ...,MA1_1A_2
1,3C,Indicate which 3TG minerals are used,MA2_3_3
2,7A,"Sort countries of origin by mineral, product, ...",MA2_6_1
3,14A,Conducted Independent Private Sector Audit,MA4_13_1
4,19B,Source from an in-region initiative not just i...,MA5_18_2


In [115]:
ind_ma1 = pd.read_csv('/Users/caged/Dropbox/RSN Mining/indicator_data_matched_with_cik_data.csv', dtype='str', usecols=['MA1_1A_2', 'cik', 'primarysymbol'])
ind_ma1 = ind_ma1.rename(columns={'MA1_1A_2': 'label'})
ind_ma1['dir_name'] = ind_ma1.apply(lambda row: ' '.join([row.primarysymbol, '--', row.cik]), axis=1)
ind_ma1['dir_path'] = ind_ma1.dir_name.apply(lambda x: os.path.join('supporting_docs_2015', x))
ind_ma1 = ind_ma1.dropna()
ind_ma1.head()

,label,cik,primarysymbol,dir_name,dir_path
0,1,66740,MMM,MMM -- 66740,supporting_docs_2015/MMM -- 66740
1,1,937966,ASML,ASML -- 937966,supporting_docs_2015/ASML -- 937966
2,0,1800,ABT,ABT -- 1800,supporting_docs_2015/ABT -- 1800
3,0,1144215,AYI,AYI -- 1144215,supporting_docs_2015/AYI -- 1144215
4,1,886125,ALU,ALU -- 886125,supporting_docs_2015/ALU -- 886125


In [116]:
# How to walk a folder
row_1 = ind_ma1.ix[0]
for file_name in os.listdir(row_1.dir_path):
    file_path = os.path.join(row_1.dir_path, file_name)
    print(file_path)

supporting_docs_2015/MMM -- 66740/0001104659-15-042396.txt
supporting_docs_2015/MMM -- 66740/a15-12384_1ex1d01.htm
supporting_docs_2015/MMM -- 66740/a15-12384_1sd.htm


In [117]:
dd = pd.read_csv('disclosure_docs_list.csv', dtype='str')
dd = dd.drop('Unnamed: 0', axis=1)
dd = dd.rename(columns={'req_cik': 'req_company_name', 'req_company_name': 'req_cik'})
dd.ret_filing_date = pd.to_datetime(dd.ret_filing_date)
dd_2015 = dd[dd.ret_filing_date.dt.year == 2015]
dd_2015.head()

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
2,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,0000706688,AARON'S INC,sdconflictminerals2014.htm,2015-05-22 12:06:05,0000706688-15-000148,SD_CONFLICT_MINERALS_2014,https://www.sec.gov/Archives/edgar/data/706688...
3,AARON'S INC,706688,http://www.sec.gov/Archives/edgar/data/706688/...,0000706688,AARON'S INC,0000706688-15-000148.txt,2015-05-22 12:06:05,0000706688-15-000148,Complete submission text file,https://www.sec.gov/Archives/edgar/data/706688...
7,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,a2014formsd.htm,2015-05-29 11:36:18,0000824142-15-000071,SD,https://www.sec.gov/Archives/edgar/data/824142...
8,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,exhibit101.htm,2015-05-29 11:36:18,0000824142-15-000071,EXHIBIT 1.01,https://www.sec.gov/Archives/edgar/data/824142...
9,AAON INC,824142,http://www.sec.gov/Archives/edgar/data/824142/...,0000824142,AAON INC,0000824142-15-000071.txt,2015-05-29 11:36:18,0000824142-15-000071,Complete submission text file,https://www.sec.gov/Archives/edgar/data/824142...


In [119]:
row_1 = ind_ma1.ix[0]
# Get just the text file
dd_2015[(dd_2015.req_cik == row_1.cik) & (dd_2015.ret_title == 'Complete submission text file')]

,req_company_name,req_cik,req_url,ret_cik,ret_company_name,ret_description,ret_filing_date,ret_sec_accession_number,ret_title,ret_url
50,3M CO,66740,http://www.sec.gov/Archives/edgar/data/66740/0...,0000066740,3M CO,0001104659-15-042396.txt,2015-06-01 15:06:30,0001104659-15-042396,Complete submission text file,https://www.sec.gov/Archives/edgar/data/66740/...


In [134]:
text_file_paths = []
list_of_labels = []
for i, row in ind_ma1.iterrows():
    results = dd_2015[(dd_2015.req_cik == row.cik) & (dd_2015.ret_title == 'Complete submission text file')].reset_index()
    if len(results) == 1:
        dir_path = row.dir_path
        text_file_name = results['ret_description'][0]
        text_file_paths.append(os.path.join(dir_path, text_file_name))
        list_of_labels.append(int(row.label))

In [135]:
list_of_text_docs = []
for file_path in text_file_paths:
    with open(file_path, 'r') as f:
        text = f.read()
    soup = BeautifulSoup(text)
    documents = soup.findAll('document')
    text_docs = ''
    for doc in documents:
        text_docs = ''.join([text_docs, doc.get_text()])
    list_of_text_docs.append(text_docs)

In [136]:
x = 75
train_data = list_of_text_docs[0:x]
train_labels = list_of_labels[0:x]
test_data = list_of_text_docs[x:]
test_labels = list_of_labels[x:]

In [137]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC

In [138]:

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_data)
X_train_counts.shape

(75, 14760)

In [139]:

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [140]:

clf = LinearSVC().fit(X_train_tfidf, train_labels)

In [141]:
X_new_counts = count_vect.transform(test_data)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [145]:
print(sum(train_labels))
print(sum(test_labels))

14
8


## Sandpit

In [46]:
from nltk.tokenize import RegexpTokenizer, SpaceTokenizer
from nltk.corpus import stopwords

# Pull out any urls
word_tokenizer = SpaceTokenizer()
simple_tokens = word_tokenizer.tokenize(documents[0].get_text())
for token in simple_tokens:
    if token.startswith('http'):
        print(token)

http://investors.3m.com/financials/sec-filings/default.aspx


In [47]:
# Don't need to do this (I think)
tokenizer = RegexpTokenizer(r'\w+')
def tkize(doc_text):
    tokens = tokenizer.tokenize(text_docs)
    tokens = [tk.lower() for tk in tokens]
    return tokens

words_list = []
for doc in documents:
    doc_text = doc.get_text()
    tks = tkize(doc_text)
    words = [w for w in tks if not w in stopwords.words("english")]
    words_list.append(words)
len(words_list)